# 감정 분석 결과를 활용한 추천 시스템

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")
# print(API_KEY)

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

from flask import Flask, request, jsonify, render_template
app = Flask(__name__)

#### 테스트용 샘플 DB 생성 

In [2]:
import sqlite3

def create_database():
    conn = sqlite3.connect("recommendations.db")
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS recommendations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            emotion TEXT,
            category TEXT,
            recommendation TEXT
        )
    """
    )
    
    data = [
        ("happy", "music", "Pharrell Williams - Happy"),
        ("happy", "music", "Bobby McFerrin - Don't Worry Be Happy"),
        ("happy", "music", "Katrina & The Waves - Walking on Sunshine"),
        ("happy", "music", "Justin Timberlake - Can't Stop The Feeling!"),
        ("happy", "music", "Taylor Swift - Shake It Off"),
        ("happy", "movies", "Inside Out"),
        ("happy", "movies", "The Pursuit of Happyness"),
        ("happy", "movies", "La La Land"),
        ("happy", "movies", "Forrest Gump"),
        ("happy", "movies", "Zootopia"),
        ("happy", "news", "World celebrates positive global changes"),
        ("happy", "news", "New scientific breakthrough cures major disease"),
        ("happy", "news", "Country achieves 100% renewable energy"),
        ("happy", "news", "Unemployment rates hit record lows"),
        ("happy", "news", "Major peace agreement signed between nations"),
        ("sad", "music", "Adele - Someone Like You"),
        ("sad", "music", "Billie Eilish - Everything I Wanted"),
        ("sad", "music", "Eric Clapton - Tears in Heaven"),
        ("sad", "music", "Sam Smith - Stay With Me"),
        ("sad", "music", "Lewis Capaldi - Someone You Loved"),
        ("sad", "movies", "The Green Mile"),
        ("sad", "movies", "Schindler's List"),
        ("sad", "movies", "Manchester by the Sea"),
        ("sad", "movies", "Requiem for a Dream"),
        ("sad", "movies", "Blue Valentine"),
        ("sad", "news", "Efforts to combat world hunger see progress"),
        ("sad", "news", "Flood victims receive unprecedented aid"),
        ("sad", "news", "Wildlife conservation efforts gain traction"),
        ("sad", "news", "New therapy gives hope to depression patients"),
        ("sad", "news", "Rescue team saves hundreds from disaster"),
        ("angry", "music", "Eminem - Lose Yourself"),
        ("angry", "music", "Linkin Park - Crawling"),
        ("angry", "music", "System of a Down - Toxicity"),
        ("angry", "music", "Rage Against the Machine - Killing in the Name"),
        ("angry", "music", "Disturbed - Down With the Sickness"),
        ("angry", "movies", "Joker"),
        ("angry", "movies", "Falling Down"),
        ("angry", "movies", "American History X"),
        ("angry", "movies", "A Clockwork Orange"),
        ("angry", "movies", "Fight Club"),
        ("angry", "news", "Protests erupt in multiple cities"),
        ("angry", "news", "Scandal exposes major corruption"),
        ("angry", "news", "Government policy sparks nationwide outrage"),
        ("angry", "news", "Human rights violations under investigation"),
        ("angry", "news", "Company faces backlash over unethical practices"),
        ("relaxed", "music", "Norah Jones - Come Away With Me"),
        ("relaxed", "music", "Coldplay - Yellow"),
        ("relaxed", "music", "Jack Johnson - Banana Pancakes"),
        ("relaxed", "music", "John Mayer - Gravity"),
        ("relaxed", "music", "Ed Sheeran - Thinking Out Loud"),
        ("relaxed", "movies", "The Secret Life of Walter Mitty"),
        ("relaxed", "movies", "Amélie"),
        ("relaxed", "movies", "Eat Pray Love"),
        ("relaxed", "movies", "Midnight in Paris"),
        ("relaxed", "movies", "Julie & Julia"),
        ("relaxed", "news", "Scientists discover new method for stress relief"),
        ("relaxed", "news", "Yoga and meditation programs gain popularity"),
        ("relaxed", "news", "Cities expanding green spaces for citizens"),
        ("relaxed", "news", "Studies show benefits of digital detoxing"),
        ("relaxed", "news", "Remote work improves mental health of employees")
    ]
    
    cursor.executemany("INSERT INTO recommendations (emotion, category, recommendation) VALUES (?, ?, ?)", data)
    conn.commit()
    conn.close()
    print("Database populated successfully!")

if __name__ == "__main__":
    create_database()

Database populated successfully!


In [3]:
import sqlite3

# def analyze_sentiment(text):
#     response = openai.ChatCompletion.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant that analyzes sentiment."},
#             {"role": "user", "content": f"Analyze the sentiment of this text: {text}"}
#         ]
#     )
#     return response['choices'][0]['message']['content'].strip()

# 감정 분석 함수
def analyze_sentiment(text):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "사용자의 감정을 분석합니다."},
            {"role": "user", "content": f"다음 문장의 감정을 분석해서 happy,sad,angry,relaxed 중에서 하나의 단어로만 말해줘\n {text}"}
        ]
    )
    return response.choices[0].message.content

# DB 검색하기 
def get_recommendations(emotion):
    conn = sqlite3.connect("recommendations.db")
    cursor = conn.cursor()
    cursor.execute("SELECT category, recommendation FROM recommendations WHERE emotion = ?", (emotion,))
    results = cursor.fetchall()
    conn.close()
    
    recommendations = {"music": [], "movies": [], "news": []}
    for category, rec in results:
        recommendations[category].append(rec)
    
    return recommendations

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    text = data.get("text", "")
    emotion = analyze_sentiment(text)
    # print(emotion)
    recommendations = get_recommendations(emotion)
    return jsonify({"emotion": emotion, "recommendations": recommendations})

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Apr/2025 16:20:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2025 16:20:51] "POST /analyze HTTP/1.1" 200 -
